In [1]:
import QuantLib as ql
from math import pow, sqrt
import numpy as np
from scipy.optimize import root

In [2]:
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

calculation_date = ql.Date(6, 11, 2015)

spot = 659.37
ql.Settings.instance().evaluationDate = calculation_date

risk_free_rate = 0.01
dividend_rate = 0.0
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))

In [3]:
expiration_dates = [ql.Date(6, 12, 2015), ql.Date(6, 1, 2016), ql.Date(6, 2, 2016),
                    ql.Date(6, 3, 2016), ql.Date(6, 4, 2016), ql.Date(6, 5, 2016),
                    ql.Date(6, 6, 2016), ql.Date(6, 7, 2016), ql.Date(6, 8, 2016),
                    ql.Date(6, 9, 2016), ql.Date(6, 10, 2016), ql.Date(6, 11, 2016),
                    ql.Date(6, 12, 2016), ql.Date(6, 1, 2017), ql.Date(6, 2, 2017),
                    ql.Date(6, 3, 2017), ql.Date(6, 4, 2017), ql.Date(6, 5, 2017),
                    ql.Date(6, 6, 2017), ql.Date(6, 7, 2017), ql.Date(6, 8, 2017),
                    ql.Date(6, 9, 2017), ql.Date(6, 10, 2017), ql.Date(6, 11, 2017)]
strikes = [527.50, 560.46, 593.43, 626.40, 659.37, 692.34, 725.31, 758.28]
data = [[0.37819, 0.34177, 0.30394, 0.27832, 0.26453, 0.25916, 0.25941, 0.26127],
        [0.3445, 0.31769, 0.2933, 0.27614, 0.26575, 0.25729, 0.25228, 0.25202],
        [0.37419, 0.35372, 0.33729, 0.32492, 0.31601, 0.30883, 0.30036, 0.29568],
        [0.37498, 0.35847, 0.34475, 0.33399, 0.32715, 0.31943, 0.31098, 0.30506],
        [0.35941, 0.34516, 0.33296, 0.32275, 0.31867, 0.30969, 0.30239, 0.29631],
        [0.35521, 0.34242, 0.33154, 0.3219, 0.31948, 0.31096, 0.30424, 0.2984],
        [0.35442, 0.34267, 0.33288, 0.32374, 0.32245, 0.31474, 0.30838, 0.30283],
        [0.35384, 0.34286, 0.33386, 0.32507, 0.3246, 0.31745, 0.31135, 0.306],
        [0.35338, 0.343, 0.33464, 0.32614, 0.3263, 0.31961, 0.31371, 0.30852],
        [0.35301, 0.34312, 0.33526, 0.32698, 0.32766, 0.32132, 0.31588, 0.31052],
        [0.35272, 0.34322, 0.33574, 0.32765, 0.32873, 0.32267, 0.31705, 0.31209],
        [0.35246, 0.3433, 0.33617, 0.32822, 0.32965, 0.32383, 0.31831, 0.31344],
        [0.35226, 0.34336, 0.33651, 0.32869, 0.3304, 0.32477, 0.31934, 0.31453],
        [0.35207, 0.34342, 0.33681, 0.32911, 0.33106, 0.32561, 0.32025, 0.3155],
        [0.35171, 0.34327, 0.33679, 0.32931, 0.3319, 0.32665, 0.32139, 0.31675],
        [0.35128, 0.343, 0.33658, 0.32937, 0.33276, 0.32769, 0.32255, 0.31802],
        [0.35086, 0.34274, 0.33637, 0.32943, 0.3336, 0.32872, 0.32368, 0.31927],
        [0.35049, 0.34252, 0.33618, 0.32948, 0.33432, 0.32959, 0.32465, 0.32034],
        [0.35016, 0.34321, 0.33602, 0.32953, 0.33498, 0.3304, 0.32554, 0.32132],
        [0.34986, 0.34213, 0.33587, 0.32957, 0.33556, 0.3311, 0.32631, 0.32217],
        [0.34959, 0.34196, 0.33573, 0.32961, 0.3361, 0.33176, 0.32704, 0.32296],
        [0.34934, 0.34181, 0.33561, 0.32964, 0.33658, 0.33235, 0.32769, 0.32368],
        [0.34912, 0.34167, 0.3355, 0.32967, 0.33701, 0.33288, 0.32827, 0.32432],
        [0.34891, 0.34154, 0.33539, 0.3297, 0.33742, 0.33337, 0.32881, 0.32492]]

In [6]:
def setup_helpers(engine, expiration_dates, strikes, data, ref_date, spot, yield_ts, dividend_ts):
    heston_helpers = []
    grid_data = []
    for i, date in enumerate(expiration_dates):
        for j, s in enumerate(strikes):
            t = (date - ref_date)
            p = ql.Period(t, ql.Days)
            vols = data[i][j]
            helper = ql.HestonModelHelper(p, calendar, spot, s, ql.QuoteHandle(ql.SimpleQuote(vols)), yield_ts, dividend_ts)
            helper.setPricingEngine(engine)
            heston_helpers.append(helper)
            grid_data.append((date, s))
    return heston_helpers, grid_data


def cost_function_generator(model, helpers, norm=False):
    def cost_function(params):
        params_ = ql.Array(list(params))
        model.setParams(params_)
        error = [h.calibrationError() for h in helpers]
        if norm:
            return np.sqrt(np.sum(np.abs(error)))
        else:
            return error
    return cost_function


def calibration_report(helpers, grid_data, detailed=False):
    avg = 0.0
    if detailed:
        print("%15s %25s %15s %15s %20s" % ("Strikes", "Expiry", "Market Value", "Model Value", "Relative Error (%)"))
        print("="*100)
    for i, opt in enumerate(helpers):
        err = (opt.modelValue() / opt.marketValue() - 1.0)
        date, strike = grid_data[i]
        if detailed:
            print("%15.2f %25s %14.5% %15.5f %20.7f" % (strike, str(date), opt.marketValue(), opt.modelValue(), 100*(opt.modelValue()/opt.marketValue()-1.0)))
        avg += abs(err)
    avg = avg*100.0/len(helpers)
    if detailed:
        print("-"*100)
    summary = "Average Abs Error (%%) : %5.9f" % (avg)
    print(summary)
    return avg


def setup_model(_yield_ts, _dividend_ts, _spot, init_condition=(0.02, 0.2, 0.5, 0.1, 0.01)):
    theta, kappa, sigma, rho, v0 = init_condition
    process = ql.HestonProcess(_yield_ts, _dividend_ts, ql.QuoteHandle(ql.SimpleQuote(_spot)), v0, kappa, theta, sigma, rho)
    model = ql.HestonModel(process)
    engine = ql.AnalyticHestonEngine(model)
    return model, engine


summary = []

In [7]:
model1, engine1 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.02, 0.2, 0.5, 0.1, 0.01))
heston_helpers1, grid_data1 = setup_helpers(engine1, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model1.params())

In [8]:
%%time
lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
model1.calibrate(heston_helpers1, lm, ql.EndCriteria(500, 300, 1.0e-8, 1.0e-8, 1.0e-8))
theta, kappa, sigma, rho, v0 = model1.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers1, grid_data1)
summary.append(["QL LM1", error] + list(model1.params()))

theta = 0.125760, kappa = 7.911958, sigma = 1.887628, rho = -0.364933, v0 = 0.055399
Average Abs Error (%) : 3.013208440
Wall time: 2.14 s


In [9]:
model1, engine1 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.07, 0.5, 0.1, 0.1, 0.1))
heston_helpers1, grid_data1 = setup_helpers(engine1, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model1.params())

In [10]:
%%time
lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
model1.calibrate(heston_helpers1, lm, ql.EndCriteria(500, 300, 1.0e-8, 1.0e-8, 1.0e-8))
theta, kappa, sigma, rho, v0 = model1.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers1, grid_data1)
summary.append(["QL LM2", error] + list(model1.params()))

theta = 0.084523, kappa = 0.000000, sigma = 0.132219, rho = -0.514027, v0 = 0.099928
Average Abs Error (%) : 11.008826759
Wall time: 2.25 s


In [11]:
model2, engine2 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.02, 0.2, 0.5, 0.1, 0.01))
heston_helpers2, grid_data2 = setup_helpers(engine2, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model2.params())

In [12]:
%%time
cost_function = cost_function_generator(model2, heston_helpers2)
sol = root(cost_function, initial_condition, method='lm')
theta, kappa, sigma, rho, v0 = model2.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers2, grid_data2)
summary.append(["SciPy LM1", error] + list(model2.params()))

theta = 0.125759, kappa = 7.912874, sigma = 1.887747, rho = -0.364933, v0 = 0.055396
Average Abs Error (%) : 3.013195727
Wall time: 2.05 s


In [13]:
model2, engine2 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.07, 0.5, 0.1, 0.1, 0.1))
heston_helpers2, grid_data2 = setup_helpers(engine2, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model2.params())

In [14]:
%%time
cost_function = cost_function_generator(model2, heston_helpers2)
sol = root(cost_function, initial_condition, method='lm')
theta, kappa, sigma, rho, v0 = model2.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers2, grid_data2)
summary.append(["SciPy LM2", error] + list(model2.params()))

theta = -42.044169, kappa = -0.000907, sigma = 0.252942, rho = -1.001501, v0 = 0.088258
Average Abs Error (%) : 5.744317043
Wall time: 22.9 s


In [15]:
from scipy.optimize import least_squares

In [20]:
model3, engine3 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.02, 0.2, 0.2, 0.1, 0.01))
heston_helpers3, grid_data3 = setup_helpers(engine3, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model3.params())

In [21]:
%%time
cost_function = cost_function_generator(model3, heston_helpers3)
sol = least_squares(cost_function, initial_condition, method='lm')
theta, kappa, sigma, rho, v0 = model3.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers3, grid_data3)
summary.append(["SciPy LS1", error] + list(model3.params()))

theta = 0.125760, kappa = 7.912208, sigma = 1.887663, rho = -0.364932, v0 = 0.055404
Average Abs Error (%) : 3.013154084
Wall time: 4.56 s


In [22]:
model3, engine3 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.07, 0.5, 0.1, 0.1, 0.1))
heston_helpers3, grid_data3 = setup_helpers(engine3, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model3.params())

In [23]:
%%time
cost_function = cost_function_generator(model3, heston_helpers3)
sol = least_squares(cost_function, initial_condition, method='lm')
theta, kappa, sigma, rho, v0 = model3.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers3, grid_data3)
summary.append(["SciPy LS2", error] + list(model3.params()))

theta = 1.116878, kappa = 0.000042, sigma = -0.000696, rho = -0.000084, v0 = 0.587810
Average Abs Error (%) : 5.098700384
Wall time: 9.08 s


In [26]:
from scipy.optimize import differential_evolution

In [27]:
model4, engine4 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.02, 0.2, 0.2, 0.1, 0.01))
heston_helpers4, grid_data4 = setup_helpers(engine4, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model4.params())
bounds = [(0, 1), (0.01, 15), (0.01, 1.0), (-1, 1), (0, 1.0)]

In [28]:
%%time
cost_function = cost_function_generator(model4, heston_helpers4, norm=True)
sol = differential_evolution(cost_function, bounds, maxiter=100)
theta, kappa, sigma, rho, v0 = model4.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers4, grid_data4)
summary.append(["SciPy DE1", error] + list(model4.params()))

theta = 0.122870, kappa = 5.109367, sigma = 0.945947, rho = -0.567321, v0 = 0.077834
Average Abs Error (%) : 2.869746009
Wall time: 1min 12s


In [29]:
model4, engine4 = setup_model(flat_ts, dividend_ts, spot)
heston_helpers4, grid_data4 = setup_helpers(engine4, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model4.params())
bounds = [(0, 1), (0.01, 15), (0.01, 1.0), (-1, 1), (0, 1.0)]

In [30]:
%%time
cost_function = cost_function_generator(model4, heston_helpers4, norm=True)
sol = differential_evolution(cost_function, bounds, maxiter=100)
theta, kappa, sigma, rho, v0 = model4.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers4, grid_data4)
summary.append(["SciPy DE2", error] + list(model4.params()))

theta = 0.122717, kappa = 4.541489, sigma = 0.747434, rho = -0.675509, v0 = 0.079010
Average Abs Error (%) : 2.881507738
Wall time: 1min


In [31]:
from scipy.optimize import basinhopping

In [32]:
class MyBounds(object):
    def __init__(self, xmin=[0., 0.01, 0.01, -1, 0], xmax=[1, 15, 1, 1, 1.0]):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin
    

bounds = [(0, 1), (0.01, 15), (0.01, 1.0), (-1, 1), (0, 1.0)]

In [38]:
model5, engine5 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.02, 0.2, 0.5, 0.1, 0.01))
heston_helpers5, grid_data5 = setup_helpers(engine5, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model5.params())

In [39]:
%%time
mybound = MyBounds()
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}
cost_function = cost_function_generator(model5, heston_helpers4, norm=True)
sol = basinhopping(cost_function, initial_condition, niter=5, minimizer_kwargs=minimizer_kwargs, stepsize=0.005, accept_test=mybound, interval=10)
theta, kappa, sigma, rho, v0 = model5.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers5, grid_data5)
summary.append(["SciPy BH1", error] + list(model5.params()))

theta = 0.024622, kappa = 0.189668, sigma = 0.501535, rho = 0.093245, v0 = 0.016846
Average Abs Error (%) : 82.692481038
Wall time: 807 ms


In [40]:
model5, engine5 = setup_model(flat_ts, dividend_ts, spot, init_condition=(0.07, 0.5, 0.1, 0.1, 0.1))
heston_helpers5, grid_data5 = setup_helpers(engine5, expiration_dates, strikes, data, calculation_date, spot, flat_ts, dividend_ts)
initial_condition = list(model5.params())

In [41]:
%%time
mybound = MyBounds()
minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}
cost_function = cost_function_generator(model5, heston_helpers4, norm=True)
sol = basinhopping(cost_function, initial_condition, niter=5, minimizer_kwargs=minimizer_kwargs, stepsize=0.005, accept_test=mybound, interval=10)
theta, kappa, sigma, rho, v0 = model5.params()
print("theta = %f, kappa = %f, sigma = %f, rho = %f, v0 = %f" % (theta, kappa, sigma, rho, v0))
error = calibration_report(heston_helpers5, grid_data5)
summary.append(["SciPy BH2", error] + list(model5.params()))

theta = 0.072327, kappa = 0.506534, sigma = 0.095887, rho = 0.090253, v0 = 0.101320
Average Abs Error (%) : 14.807595395
Wall time: 763 ms


In [42]:
import pandas as pd
pd.DataFrame(summary, columns=["Name", "Avg Abs Error", "Theta", "Kappa", "Sigma", "Rho", "V0"], index=['']*len(summary))

,Name,Avg Abs Error,Theta,Kappa,Sigma,Rho,V0
,QL LM1,3.013208,0.125760,7.911958e+00,1.887628,-0.364933,0.055399
,QL LM1,11.008827,0.084523,1.702329e-08,0.132219,-0.514027,0.099928
,SciPy LM1,3.013196,0.125759,7.912874e+00,1.887747,-0.364933,0.055396
,SciPy LM2,5.744317,-42.044169,-9.067011e-04,0.252942,-1.001501,0.088258
,SciPy LS1,3.013209,0.125760,7.912104e+00,1.887651,-0.364932,0.055399
,SciPy LS2,5.744317,-42.044169,-9.067011e-04,0.252942,-1.001501,0.088258
,SciPy LS1,3.013154,0.125760,7.912208e+00,1.887663,-0.364932,0.055404
,SciPy LS2,5.098700,1.116878,4.171596e-05,-0.000696,-0.000084,0.587810
,SciPy DE1,2.869746,0.122870,5.109367e+00,0.945947,-0.567321,0.077834
,SciPy DE2,2.881508,0.122717,4.541489e+00,0.747434,-0.675509,0.079010
